## Dùng nn.Module

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [27]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()     #gọi hàm khởi tạo của nn.module
        self.conv1 = nn.Conv2d(1,6,3)   #(input_channel, output_channel, kernel)
        self.conv2 = nn.Conv2d(6,16,3)
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2,2))      # (2,2) kernel của maxpooling
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  

        x = x.view(-1, self.num_flat_features(x))   # chuyển chiều ma trận với hàm

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        return x
    
    def num_flat_features(self, x):
        # bỏ phần demension đầu tiên và nhân các demension còn lại với nhau
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    

In [28]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [29]:
input_image = torch.rand(1, 1, 32, 32)  # (  , channel, w, h)
output = net(input_image)
output.size()

torch.Size([1, 10])

## Dùng nn.Sequential

In [30]:
import torch
import torch.nn as nn

In [31]:
net = nn.Sequential()

In [32]:
class Flatten(nn.Module):
    def forward(self, x):
        size = x.size()[1:]     # VD: x = (10,3,32,32)
        num_features = 1
        for s in size:
            num_features *= s
        return x.view(-1, num_features)     # x = (10, 3*32*32)


In [33]:
net.add_module("Conv1", nn.Conv2d(1,6,3))
net.add_module("ReLU1", nn.ReLU())
net.add_module("Maxpooling1", nn.MaxPool2d(2))

net.add_module("Conv2", nn.Conv2d(6,16,3))
net.add_module("ReLU2", nn.ReLU())
net.add_module("Maxpooling2", nn.MaxPool2d(2))

net.add_module("Flatten", Flatten())

net.add_module("Fc1", nn.Linear(16*6*6, 120))
net.add_module("Fc2", nn.Linear(120, 84))
net.add_module("Fc3", nn.Linear(84, 10))

In [34]:
print(net)

Sequential(
  (Conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (ReLU1): ReLU()
  (Maxpooling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (ReLU2): ReLU()
  (Maxpooling2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Flatten): Flatten()
  (Fc1): Linear(in_features=576, out_features=120, bias=True)
  (Fc2): Linear(in_features=120, out_features=84, bias=True)
  (Fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [35]:
input = torch.rand(1, 1, 32, 32)

In [36]:
output = net(input)

In [37]:
print(output.shape)

torch.Size([1, 10])
